In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
!pip install transformers

# 필요한 라이브러리 불러오기

In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# BERT 입력(Tokenizer 부분)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = 256

In [ ]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []
  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=  max_seq_len, pad_to_max_length=True)
    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  return (input_ids, attention_masks, token_type_ids)

# 실제 데이터 구현해보기

In [ ]:
# # 원본
# class TFBertForSequenceClassification(tf.keras.Model):
#   def __init__(self, model_name, num_labels):
#     super(TFBertForSequenceClassification, self).__init__()
#     self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
#     self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
#                                             activation='softmax',name='classifier')


#   def call(self, inputs):
#     input_ids, attention_mask, token_type_ids = inputs
#     outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
#     token_type_ids=token_type_ids)
#     cls_token = outputs[1]
#     prediction = self.classifier(cls_token)
#     return prediction

In [ ]:
# additional layer
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                            activation='softmax',name='classifier')
    # 추가 된 것
    self.additional_layer = tf.keras.layers.Dense(
        128, #128 유닛을 가진 레이어 추가 (변경가능)
        activation ='relu'
    )


  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
    token_type_ids=token_type_ids)
    cls_token = outputs[1]

    # 추가적인 레이어를 거쳐서 분류 수행
    hidden = self.additional_layer(cls_token)
    prediction = self.classifier(cls_token)
    return prediction

## TPU 작동

In [ ]:
# TPU 작  동  을     위  한       코  드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base", num_labels=2)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])
  # metrics 인자에 ['accuracy', precision_score, recall_score, f1_score]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

## 예측할 데이터셋 만들기

In [ ]:
#  실제 데이터 해보기
import json
import os
import csv
import nltk
from transformers import BertTokenizer

# nltk 라이브러리에서 문장 토크나이저를 사용
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# BERT tokenizer 불러오기
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

text = '''난임을 고백한 이효리가 의미심장한 발언을 해 사람들의 궁금증을 자아내고 있는 가운데, 최근 '캐나다 체크인'에서 2세 계획을 공개하며 많은 주목을 받았습니다.

2022년 12월 31일에 방송된 tvN 예능 '캐나다 체크인'에는 이효리가 캐나다에서 '링고' 집에 머무르는 이야기가 담겼습니다. ‘캐나다 체크인’은 10년 넘게 유기견 봉사를 해온 이효리가 자신이 보살피던 중 해외의 새로운 가족에게 입양 보낸 개들을 만나기 위해 캐나다로 떠나는 여정을 담은 프로그램입니다.

링고는 이효리가 제주 김녕 바닷가에서 우연히 발견해 임시보호 하다 입양보낸 반려견입니다. 이날 방송에서 이효리는 화목한 '링고' 가족을 본 뒤 자신의 가족을 떠올렸습니다.

그는 "엄마한테 연락을 자주 해야 하는데 잘 안하게 된다. 힘든 이야기만 하니까"라며 투병 중인 아버지와 간병 중인 어머니에 대해 고백했습니다.

이효리는 함께 온 '공길 언니' 고인숙씨에게 "언니, 제주도에서 우리 아빠 동영상 봤었잖아. 나는 아빠랑 별로 친하지도 않았는데 눈물이 나더라"고 말문을 텄습니다.

그러자 인숙씨는 "너 없을 때 아버지께 '효리 어렸을 때 어땠어요'라고 물으니 아버지가 되게 자랑스러워 하시더라"라며 아버지와 대화를 들려줬습니다.

이에 이효리는 "맞아 아버지가 나는 안 잊었어"라고 하자 인숙씨는 "너는 그걸로 효도를 다 한거지"라고 답했습니다.

이날 이효리는 방송에서 첫 번째 반려견으로 그의 곁을 떠난 순심이와 똑 닮은 강아지를 보고 눈물짓기도 했습니다.

서핑 성지로 불리는 캐나다 밴쿠버 섬 서부 해안의 토피노에서 순심이와 닮은 개를 본 이효리는 "순심이랑 닮았다. 털 색깔도 똑같다"라고 말하며 눈물을 보였습니다.

그는 "진짜 똑같이 생겼다. 아 진짜 못 살겠다. 내가 말했잖아 어딘가에서 똑같이 살고 있을 거 같다고. 저게 그건가. 몇 년이 지나도 어떤 포인트만 딱 건드리면 그냥 눈물이 난다"라며 "강형욱 씨가 말했잖아. 펫로스 증후군은 극복 못했다고. 그건 극복할 수 있는 게 아닌 거 같다. 극복할 수 없다"며 순심이를 향한 애틋한 감정을 드러냈습니다

또한 이날 방송에서 이효리는 연예인이 아닌 인간 이효리에 대해 입을 열었습니다. 이효리는 “20살로 돌아간다면 연예인을 하겠냐”는 질문에 “아니다. 평범하게 살 것 같다”고 대답했습니다.

이효리는 “캐나다에 와서 더 느꼈다. 결혼해서 아이 낳고, 가족을 이루고, 아이를 키우면서 그렇게 살고 싶다”고 덧붙였습니다. 이어 이효리는 해외입양을 보낸 강아지 링고네를 찾은 뒤 할머니부터 손녀까지 살고 있는 대가족 분위기에 부러움을 드러냈습니다.

이어 “여기 캐나다에 와서 전체적인 걸 보고서 되게 가족 중심이고 뭔가 차분하고 에너지 같은 걸 뺏기는 게 아니라 고요히 간직하고 있는 느낌이라 해야 되나, 사람들이 막 휘둘리는 느낌이 아니고”라고 덧붙였습니다. 또한 이효리는 연예인으로 사는 고충을 털어놓았습니다.


그는 “이제 연예인 하면서는 휘둘릴 수밖에 없는 게 많으니까. 나는 그게 제일 어려운 거 같다. 사람들이 나를 모두 손가락질하고 비난한다고 생각만 해도 심장이 막 떨려. 그래서 이 생활이 맞나 틀리나 헷갈린다”라고 토로했습니다.

이에 지인은 “너무 잘하고 있다. 매번 사람들에게 새로운 영향을 주잖아”라고 말했고, 이효리는 “근데 어떤 사람한테 영향을 준다는 자체가 너무 무섭지 않아? 나도 계속 변하고 생각도 변하고, 근데 한 가지로 규정지어 버리면”이라고 두려운 마음을 전했습니다

한편 이효리 지난 2013년 기타리스트 이상순과 결혼해 제주도에 살고 있습니다. 유기견 봉사활동을 하면서 남편 이상순과 9년의 결혼 생활을 했지만 현재 2세 소식이 없어 많은 사람들이 궁금해 하고 있습니다. 이에 이효리는 지난 4월 티빙 예능 ‘서울체크인’에서 2세에 대한 고민을 토로했습니다.

이효리는 김종민, 신지, 은지원, 딘딘과 스키장으로 여행을 떠나 숙소에서 뒤풀이 중 속내를 털어놨습니다. 김종민은 "너는 아기 생각 없어?"라고 조심스럽게 물어봤고 이효리는 "생각이 있다. 근데 이게 생각보다 쉽지가 않다. 나이가 있어서 그런 것 같다"고 말했습니다.

또한 서울체크인에서 이효리는 가수 비와 대화를 하면서 "아기들도 예쁘게 잘 크고 있지 않느냐"면서 부러워하는 모습을 보였습니다. "스타킹도 사고 시도도 한다. 근데 오빠랑 너무 24시간 붙어 있다"고 불만을 토로했습니다.

이번 '캐나다 체크인'에서 이효리는 대가족을 보면서 “나는 오빠랑 둘이 살잖아. 되게 부럽더라”고 털어놨습니다. 그러면서 이효리는 아들 셋을 둔 지인에게 “비법 좀 알려줘”라고 물었고, 지인은 “남편을 뜨겁게 사랑하면 돼”라고 팁을 전수했습니다.

이에 이효리는 “언니는 허니문 때 처음 가졌잖아. 허니문 때는 격하게 사랑할 수 있지. 아주 불타오르지”라고 말해 웃음을 자아냈습니다.

이효리가 난임에 대해 언급하면서 남편 이상순과의 입맞춤에 대해 솔직하게 털어놨습니다. 2022년 6월 3일 공개된 티빙 '서울체크인' 7회에서 이상순이 서울에 와서 이효리와 데이트를 했습니다.

이날 이상순은 "오늘 홍현희씨 집에서 자는거냐"고 물었고 이효리는 "그렇다"며 "홍현희와 제이쓴은 아직도 뽀뽀하고 알콩달콩하다"고 덧붙였습니다.

이에 이상순은 "우리도 하면 되지 않나, 뽀뽀 한번 할까"라고 분위기를 잡았습니다. 이효리는 이런 이상순을 보고 웃은 뒤 "하자 그냥"이라며 입을 맞췄지만 이후 이효리는 "아무 감흥이 없다. 내 팔에 뽀뽀하는 기분"이라고 밝혔습니다.

이에 이상순은 "그건 좀 심하지 않나"라며 서운해 했고 이효리는 "뽀뽀하는건 좋은데 키스는 싫다"고 덧붙였습니다. 이상순은 "싫을 것까지는 없지 않냐"고 토로하며 서운한 기색을 드러냈습니다.

이효리는 "자고 일어났을 때, 밥 먹고 나서는 싫다"고 했고, 이상순은 "그때는 나도 안하지"라고 답했으며 이효리는 "옛날에 연애할 때는 입냄새도 안났던 것 같다"고 덧붙였습니다. 이상순은 "시간이 지나서 느낌은 다르지만 지금 더 좋은 것이 있다"고 애정을 드러냈습니다. 이효리는 "그렇다, 하루하루 정말 재미있다"고 말했습니다.
'''

with open("/content/drive/MyDrive/KOBERT/text_test.csv", 'w', newline='', encoding='utf-8') as csv_file:
  csv_writer = csv.writer(csv_file)
  csv_writer.writerow(['sent_num1', 'sent_num2', 'sentence1', 'sentence2'])



  # 문장 토크나이징
  sentences = sent_tokenize(text)
  print(sentences)

  for idx in range(len(sentences) - 1):  # 마지막 문장 제외

    sent_num1 = idx + 1
    sent_num2 = idx + 2
    sentence1 = sentences[idx]
    sentence2 = sentences[idx + 1]
    row = [sent_num1, sent_num2, sentence1, sentence2]
    csv_writer.writerow(row)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


["난임을 고백한 이효리가 의미심장한 발언을 해 사람들의 궁금증을 자아내고 있는 가운데, 최근 '캐나다 체크인'에서 2세 계획을 공개하며 많은 주목을 받았습니다.", "2022년 12월 31일에 방송된 tvN 예능 '캐나다 체크인'에는 이효리가 캐나다에서 '링고' 집에 머무르는 이야기가 담겼습니다.", '‘캐나다 체크인’은 10년 넘게 유기견 봉사를 해온 이효리가 자신이 보살피던 중 해외의 새로운 가족에게 입양 보낸 개들을 만나기 위해 캐나다로 떠나는 여정을 담은 프로그램입니다.', '링고는 이효리가 제주 김녕 바닷가에서 우연히 발견해 임시보호 하다 입양보낸 반려견입니다.', "이날 방송에서 이효리는 화목한 '링고' 가족을 본 뒤 자신의 가족을 떠올렸습니다.", '그는 "엄마한테 연락을 자주 해야 하는데 잘 안하게 된다.', '힘든 이야기만 하니까"라며 투병 중인 아버지와 간병 중인 어머니에 대해 고백했습니다.', '이효리는 함께 온 \'공길 언니\' 고인숙씨에게 "언니, 제주도에서 우리 아빠 동영상 봤었잖아.', '나는 아빠랑 별로 친하지도 않았는데 눈물이 나더라"고 말문을 텄습니다.', '그러자 인숙씨는 "너 없을 때 아버지께 \'효리 어렸을 때 어땠어요\'라고 물으니 아버지가 되게 자랑스러워 하시더라"라며 아버지와 대화를 들려줬습니다.', '이에 이효리는 "맞아 아버지가 나는 안 잊었어"라고 하자 인숙씨는 "너는 그걸로 효도를 다 한거지"라고 답했습니다.', '이날 이효리는 방송에서 첫 번째 반려견으로 그의 곁을 떠난 순심이와 똑 닮은 강아지를 보고 눈물짓기도 했습니다.', '서핑 성지로 불리는 캐나다 밴쿠버 섬 서부 해안의 토피노에서 순심이와 닮은 개를 본 이효리는 "순심이랑 닮았다.', '털 색깔도 똑같다"라고 말하며 눈물을 보였습니다.', '그는 "진짜 똑같이 생겼다.', '아 진짜 못 살겠다.', '내가 말했잖아 어딘가에서 똑같이 살고 있을 거 같다고.', '저게 그건가.', '몇 년이 지나도 어떤 포인트만 딱 건드리면 그냥 눈물이 난다"라며 "강

In [ ]:
import pandas as pd
csv_file_path_test = '/content/drive/MyDrive/KOBERT/text_test.csv'
test= pd.read_csv(csv_file_path_test,encoding='utf-8',header=0)


X1_test = convert_examples_to_features(test['sentence1'], test['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)
test


  0%|          | 0/62 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

100%|██████████| 62/62 [00:00<00:00, 365.20it/s]


,sent_num1,sent_num2,sentence1,sentence2
0,1,2,난임을 고백한 이효리가 의미심장한 발언을 해 사람들의 궁금증을 자아내고 있는 가운데...,2022년 12월 31일에 방송된 tvN 예능 '캐나다 체크인'에는 이효리가 캐나다...
1,2,3,2022년 12월 31일에 방송된 tvN 예능 '캐나다 체크인'에는 이효리가 캐나다...,‘캐나다 체크인’은 10년 넘게 유기견 봉사를 해온 이효리가 자신이 보살피던 중 해...
2,3,4,‘캐나다 체크인’은 10년 넘게 유기견 봉사를 해온 이효리가 자신이 보살피던 중 해...,링고는 이효리가 제주 김녕 바닷가에서 우연히 발견해 임시보호 하다 입양보낸 반려견입니다.
3,4,5,링고는 이효리가 제주 김녕 바닷가에서 우연히 발견해 임시보호 하다 입양보낸 반려견입니다.,이날 방송에서 이효리는 화목한 '링고' 가족을 본 뒤 자신의 가족을 떠올렸습니다.
4,5,6,이날 방송에서 이효리는 화목한 '링고' 가족을 본 뒤 자신의 가족을 떠올렸습니다.,"그는 ""엄마한테 연락을 자주 해야 하는데 잘 안하게 된다."
...,...,...,...,...
57,58,59,"내 팔에 뽀뽀하는 기분""이라고 밝혔습니다.","이에 이상순은 ""그건 좀 심하지 않나""라며 서운해 했고 이효리는 ""뽀뽀하는건 좋은데..."
58,59,60,"이에 이상순은 ""그건 좀 심하지 않나""라며 서운해 했고 이효리는 ""뽀뽀하는건 좋은데...","이상순은 ""싫을 것까지는 없지 않냐""고 토로하며 서운한 기색을 드러냈습니다."
59,60,61,"이상순은 ""싫을 것까지는 없지 않냐""고 토로하며 서운한 기색을 드러냈습니다.","이효리는 ""자고 일어났을 때, 밥 먹고 나서는 싫다""고 했고, 이상순은 ""그때는 나..."
60,61,62,"이효리는 ""자고 일어났을 때, 밥 먹고 나서는 싫다""고 했고, 이상순은 ""그때는 나...","이상순은 ""시간이 지나서 느낌은 다르지만 지금 더 좋은 것이 있다""고 애정을 드러냈..."


## Checkpoint에서 pretrained Model 로딩하기

In [ ]:
from tensorflow.keras.models import load_model
checkpoint_path = '/content/drive/MyDrive/KOBERT/klue_check/kluecp_add.h5'

# 저장된 가중치를 로드하여 모델 생성
loaded_model = TFBertForSequenceClassification("klue/bert-base", num_labels=2)
loaded_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
loaded_model(X1_test) # ()에 예측하고 싶은 테스트셋 넣으세요.
loaded_model.load_weights(checkpoint_path)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

## 모델 사용해서 결과값 확인하기

In [ ]:

# 로드된 모델 사용
predictions = loaded_model.predict(X1_test, batch_size=8)
# 실제 예측된 클래스 값을 확인
predicted_classes = np.argmax(predictions, axis=1)

print("Predicted classes:", predicted_classes)


8/8 [==============================] - 17s 1s/step
Predicted classes: [1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1]


In [ ]:
import pandas as pd
csv_file_path_test = '/content/drive/MyDrive/KOBERT/text_test.csv'
test= pd.read_csv(csv_file_path_test,encoding='utf-8',header=0)


test['gold_label'] = list(predicted_classes)
test


,sent_num1,sent_num2,sentence1,sentence2,gold_label
0,1,2,난임을 고백한 이효리가 의미심장한 발언을 해 사람들의 궁금증을 자아내고 있는 가운데...,2022년 12월 31일에 방송된 tvN 예능 '캐나다 체크인'에는 이효리가 캐나다...,1
1,2,3,2022년 12월 31일에 방송된 tvN 예능 '캐나다 체크인'에는 이효리가 캐나다...,‘캐나다 체크인’은 10년 넘게 유기견 봉사를 해온 이효리가 자신이 보살피던 중 해...,1
2,3,4,‘캐나다 체크인’은 10년 넘게 유기견 봉사를 해온 이효리가 자신이 보살피던 중 해...,링고는 이효리가 제주 김녕 바닷가에서 우연히 발견해 임시보호 하다 입양보낸 반려견입니다.,1
3,4,5,링고는 이효리가 제주 김녕 바닷가에서 우연히 발견해 임시보호 하다 입양보낸 반려견입니다.,이날 방송에서 이효리는 화목한 '링고' 가족을 본 뒤 자신의 가족을 떠올렸습니다.,1
4,5,6,이날 방송에서 이효리는 화목한 '링고' 가족을 본 뒤 자신의 가족을 떠올렸습니다.,"그는 ""엄마한테 연락을 자주 해야 하는데 잘 안하게 된다.",0
...,...,...,...,...,...
57,58,59,"내 팔에 뽀뽀하는 기분""이라고 밝혔습니다.","이에 이상순은 ""그건 좀 심하지 않나""라며 서운해 했고 이효리는 ""뽀뽀하는건 좋은데...",1
58,59,60,"이에 이상순은 ""그건 좀 심하지 않나""라며 서운해 했고 이효리는 ""뽀뽀하는건 좋은데...","이상순은 ""싫을 것까지는 없지 않냐""고 토로하며 서운한 기색을 드러냈습니다.",1
59,60,61,"이상순은 ""싫을 것까지는 없지 않냐""고 토로하며 서운한 기색을 드러냈습니다.","이효리는 ""자고 일어났을 때, 밥 먹고 나서는 싫다""고 했고, 이상순은 ""그때는 나...",1
60,61,62,"이효리는 ""자고 일어났을 때, 밥 먹고 나서는 싫다""고 했고, 이상순은 ""그때는 나...","이상순은 ""시간이 지나서 느낌은 다르지만 지금 더 좋은 것이 있다""고 애정을 드러냈...",1


In [ ]:
num_related = np.sum(predicted_classes == 1)
# len(predicted_classes)
relation_prop = (num_related / len(predicted_classes) )*100
print(f"이 기사의 일관성이 {(relation_prop):.2f}% 일치합니다.")


이 기사의 일관성이 0.81% 일치합니다.
